In [1]:
import bo_methods_lib
from bo_methods_lib.CS2_bo_functions_multi_dim import LHS_Design, test_train_split
from bo_methods_lib.CS2_create_data import create_y_data, calc_y_exp, create_sse_data, calc_muller
from bo_methods_lib.bo_functions_generic import clean_1D_arrays

from scipy.stats import qmc
import torch
import numpy as np
import pandas as pd
import os

/afs/crc.nd.edu/user/m/mcarlozo/.conda/envs/Toy_Problem_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#LHS Design in python - May switch code to use this instead for conciseness
emulator = True
# emulator = False
seed = 9
# LHS_Points = 100
LHS_Points =20
d = 8
Bound_Cut = True

if Bound_Cut == True:
    cut_bounds = '_cut_bounds'
else:
    cut_bounds = ""
        
# d = 2
sampler = qmc.LatinHypercube(d=d, seed = seed)
sample = sampler.random(n=LHS_Points)

#Only going to regress "a and b" parameters
skip_param_types = 1 #(Skiiping A)
l_bounds =  [-2, -2, -10, -2, -2, -2,  5, -2]
u_bounds =  [ 2,  2,   0,  2,  2,  2, 15,  2]

# l_bounds =  [-2, -2, -10, -2]
# u_bounds =  [ 2,  2,   0,  2]

# l_bounds =  [-2, -2]
# u_bounds =  [2, 2 ]
LHS = qmc.scale(sample, l_bounds, u_bounds)

In [3]:
#LHS for experimental data points
exp_LHS_Points = 20 #0 , 10, 15, 20, or 25 for 4, 8, 12, 16, and 24 params
exp_d = 2
exp_sampler = qmc.LatinHypercube(d=exp_d, seed = seed)
exp_sample = exp_sampler.random(n=exp_LHS_Points)
exp_sample_corner = exp_sampler.random(n=exp_LHS_Points)

if Bound_Cut == True:
    exp_l_bounds =  [-1.0, 0.0]
    exp_u_bounds =  [0.5 , 1.5]
else:
    exp_l_bounds =  [-1.5, -0.5]
    exp_u_bounds =  [1 , 2]

exp_l_bounds_corner =  [0, 1]
exp_u_bounds_corner =  [1 , 2]

exp_LHS = qmc.scale(exp_sample, exp_l_bounds, exp_u_bounds)
exp_LHS_corner = qmc.scale(exp_sample_corner, exp_l_bounds_corner, exp_u_bounds_corner)
#Add Extra experimental data from corner LHS
# exp_LHS = np.concatenate((exp_LHS, exp_LHS_corner))

# print(exp_LHS)
# print(exp_LHS_corner)

print(exp_LHS)

[[-0.84026869  0.12848871]
 [ 0.37976389  0.01668494]
 [-0.4537056   0.90634649]
 [ 0.13547048  1.28113218]
 [-0.85199408  1.0922064 ]
 [ 0.31362918  0.52011344]
 [-0.32542195  0.68770339]
 [-0.17374767  1.21615426]
 [-0.19867026  0.39709997]
 [-0.72243858  0.76944386]
 [-0.04598104  0.31630568]
 [-0.69908056  0.60103211]
 [-0.99121469  0.83153925]
 [-0.60311552  1.00841273]
 [ 0.43075713  0.29327019]
 [-0.50275987  1.37955458]
 [ 0.01417026  1.14053566]
 [ 0.26036852  1.49533099]
 [-0.25948888  0.15279138]
 [ 0.11504191  0.58304263]]


In [4]:
#Creates Experimental data and saves to a csv
# Constants = np.array([-200,-100,-170,15,   #A
#                       -1,-1,-6.5,0.7,    #a
#                       0,0,11,0.6,        #b
#                       -10,-10,-6.5,0.7,  #c
#                       1,0,-0.5,-1,      #x0
#                       0,0.5,1.5,1])     #yo

Constants = np.array([[-200,-100,-170,15],
                      [-1,-1,-6.5,0.7],
                      [0,0,11,0.6],
                      [-10,-10,-6.5,0.7],
                      [1,0,-0.5,-1],
                      [0,0.5,1.5,1]])

# Theta_True = np.array([-2,2])
# Xexp = np.array([-2,-1,0,1,2])
# Xexp = np.array([[-0.558,-0.050,0.623,-0.82, 0.22],
#                  [1.442,0.467,0.028,0.62,0.30]]) #This one doesn't have corner points
Xexp = np.array([[-0.558,-0.050,0.623,-0.82, 0.22],
                 [1.442,0.467,0.028,0.62,0.30]])

Xexp = np.vstack((exp_LHS, Xexp.T))
print(Xexp[0])
print(Xexp.shape)

n = Xexp.shape[0]
print(n)
noise_std = 0.01
# noise_std = 0
Yexp = calc_y_exp(Constants, Xexp, noise_std)
print(Yexp)

[-0.84026869  0.12848871]
(25, 2)
25
[   5.68740912  -94.55260334  -10.12209502   45.16919934 -111.02282502
  -58.2091376   -44.48559388    9.04355908  -73.60027422  -35.85895871
  -69.23110898  -43.98915909  -69.05884868  -46.05774234  -66.7652291
 -133.20342173   30.65841012   74.93721233  -35.43946006  -64.55771756
 -146.69947812  -80.76773816 -108.16663897  -40.67072465  -72.22508357]


In [5]:
df = pd.DataFrame({'X1exp': Xexp[:, 0], 'X2exp': Xexp[:, 1], 'Yexp':Yexp})
# df = pd.DataFrame({'Xexp': Xexp, 'Yexp':Yexp})
print(df)

path = 'Input_CSVs/Exp_Data/d='+str(exp_d)
if not os.path.exists(path):
    os.makedirs(path)
df.to_csv(os.path.join(path,'n='+str(n)+cut_bounds+'.csv'))
# df.to_csv(os.path.join(path,'n='+str(n)+'_no_noise.csv'))

       X1exp     X2exp        Yexp
0  -0.840269  0.128489    5.687409
1   0.379764  0.016685  -94.552603
2  -0.453706  0.906346  -10.122095
3   0.135470  1.281132   45.169199
4  -0.851994  1.092206 -111.022825
5   0.313629  0.520113  -58.209138
6  -0.325422  0.687703  -44.485594
7  -0.173748  1.216154    9.043559
8  -0.198670  0.397100  -73.600274
9  -0.722439  0.769444  -35.858959
10 -0.045981  0.316306  -69.231109
11 -0.699081  0.601032  -43.989159
12 -0.991215  0.831539  -69.058849
13 -0.603116  1.008413  -46.057742
14  0.430757  0.293270  -66.765229
15 -0.502760  1.379555 -133.203422
16  0.014170  1.140536   30.658410
17  0.260369  1.495331   74.937212
18 -0.259489  0.152791  -35.439460
19  0.115042  0.583043  -64.557718
20 -0.558000  1.442000 -146.699478
21 -0.050000  0.467000  -80.767738
22  0.623000  0.028000 -108.166639
23 -0.820000  0.620000  -40.670725
24  0.220000  0.300000  -72.225084


In [6]:
#Array for Standard Approach
all_p_standard = pd.DataFrame(LHS, columns = ["a1", "a2", "a3", "a4", "b1", "b2", "b3", "b4"])
# all_p_standard = pd.DataFrame(LHS, columns = ["a1", "a2", "a3", "a4"])
# all_p_standard = pd.DataFrame(LHS, columns = ["theta1", "theta2"])

print(Xexp,Xexp.shape)
#Array for Emulator approach
df = all_p_standard.iloc[np.repeat(np.arange(len(all_p_standard)), n)]
df = df.reset_index(drop = True)
# Xexp_repeat = np.repeat(Xexp, LHS_Points, axis = 1).reshape(Xexp.shape[1],-1)
# print(Xexp_repeat.reshape(300,2))
Xexp_repeat = np.tile(Xexp, (len(all_p_standard),1))
#Add Xexp 
for i in range(Xexp.shape[1]):
    df['X'+str(i+1)+'exp'] = Xexp_repeat[:,i]    
all_p_emulator = df

# print(all_p_emulator)
# print(all_p_standard)


[[-0.84026869  0.12848871]
 [ 0.37976389  0.01668494]
 [-0.4537056   0.90634649]
 [ 0.13547048  1.28113218]
 [-0.85199408  1.0922064 ]
 [ 0.31362918  0.52011344]
 [-0.32542195  0.68770339]
 [-0.17374767  1.21615426]
 [-0.19867026  0.39709997]
 [-0.72243858  0.76944386]
 [-0.04598104  0.31630568]
 [-0.69908056  0.60103211]
 [-0.99121469  0.83153925]
 [-0.60311552  1.00841273]
 [ 0.43075713  0.29327019]
 [-0.50275987  1.37955458]
 [ 0.01417026  1.14053566]
 [ 0.26036852  1.49533099]
 [-0.25948888  0.15279138]
 [ 0.11504191  0.58304263]
 [-0.558       1.442     ]
 [-0.05        0.467     ]
 [ 0.623       0.028     ]
 [-0.82        0.62      ]
 [ 0.22        0.3       ]] (25, 2)


In [7]:
## Create Training Data and Testing Data
#Set noise parameters and load csv file to generate training data
# all_p = LHS_Design("LHS_Toy_3_Input.csv")

#Create y or SSE data
#Save data
if emulator == False:
    all_2_data = all_p_standard.copy()
    all_2_in_y = create_sse_data(all_p_standard, Xexp, Yexp, Constants, obj = "obj", skip_param_types=skip_param_types)
    all_2_data['Yexp'] = all_2_in_y
#     
    all_2_data_ln_obj = all_p_standard.copy()
    all_2_in_y_ln_obj = create_sse_data(all_p_standard, Xexp, Yexp, Constants, obj = "LN_obj", skip_param_types=skip_param_types)
    all_2_data_ln_obj['Yexp'] = all_2_in_y_ln_obj
    
    titles_all = np.array(['all_st_data','all_st_ln_obj_data'])
    len_data = str(len(all_2_data))
    all_array = [all_2_data,all_2_data_ln_obj]

else:
    all_3_data = all_p_emulator.copy()
    all_3_in_y = create_y_data(all_p_emulator, Constants, Xexp, skip_param_types=skip_param_types)
    all_3_data['Yexp'] = all_3_in_y
    
    titles_all = np.array(['all_emul_data'])
    len_data = str(len(all_3_data))
    all_array = [all_3_data]
    
# print(all_array)

# Save Train Data/Test Data
for i in range(len(titles_all)):
    path = 'Input_CSVs/Train_Data/d=' + str(d) +"/"+ titles_all[i]+"/"
    if not os.path.exists(path):
        os.makedirs(path)
    all_array[i].to_csv(os.path.join(path,'t='+len_data + cut_bounds)+'.csv')